In [3]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

In [4]:
!sudo apt-get install -qq w3m # to act as web browser
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

/content
mkdir: cannot create directory ‘drive’: File exists
/content/drive
mkdir: cannot create directory ‘MyDrive’: File exists
/content
/
fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


### Init Values

In [29]:
import keras

train_percentaje = 1
validation_percentaje = 0
test_percentaje = 0
epochs_ = 1000

genres = ['blues', 'classical', 'country', 'disco',
          'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

### Select data for each model

#### Global get_data

In [6]:
import numpy as np

def index_genre(genre, genres):
    for (g,index) in zip(genres, range(len(genres))):
        if(g == genre):
            return index
    return -1

def get_data(data_path, genres, decoder, training_percentaje=0.6, validation_percentaje=0.2, test_percentaje=0.2):
    """
    data_path: se le pasa la direccion de la carpeta donde se encuentra la base de datos.
    genres: se le pasa una lista con los nombres da cada carpeta que contiene un genero dado.
    decoder: funcion para decodificar el dato que se le pasa, por ejemplo en caso de imagenes habria hacer imread
    """

    data_training = {'in': [], 'out': []}
    data_validation = {'in': [], 'out': []}
    data_test = {'in': [], 'out': []}

    for genre in genres:
        files = os.listdir(data_path + genre)
        count = len(files)

        for (filename, index) in zip(files, range(len(files))):
            filepath = data_path + genre + '/' + filename

            if (index < training_percentaje * count):
                data_training['in'].append(decoder(filepath))
                data_training['out'].append(index_genre(genre, genres))
                continue

            if (index < (training_percentaje + validation_percentaje) * count):
                data_validation['in'].append(decoder(filepath))
                data_validation['out'].append(index_genre(genre, genres))
            else:
                data_test['in'].append(decoder(filepath))
                data_test['out'].append(index_genre(genre, genres))

    data_training = {'in': np.array(data_training['in']),'out': np.array(data_training['out'])}
    data_validation = {'in': np.array(data_validation['in']),'out': np.array(data_validation['out'])}
    data_test = {'in': np.array(data_test['in']),'out': np.array(data_test['out'])}

    return {
        'data_training': data_training,
        'data_validation': data_validation,
        'data_testing': data_test
    }

#### Get data for MFCC model

In [ ]:
import cv2
import os

size_images = (192, 256)

def read_image(filepath):
    global size_images
    image = cv2.imread(filepath)
    return cv2.resize(image, size_images)


def get_data_mfcc(training_percentaje=0.8, validation_percentaje=0.1, test_percentaje=0.1, data_path = '/content/drive/MyDrive/GTZAN/Ensemble_Data/MFCC/ensemble/'):
    return get_data(data_path, genres, read_image,training_percentaje,validation_percentaje,test_percentaje)

mfcc_data = get_data_mfcc(1,0,0, '/content/drive/MyDrive/GTZAN/Ensemble_Data/MFCC/ensemble/')

temp  = get_data_mfcc(0.7, 0.15, 0.15, '/content/drive/MyDrive/GTZAN/Ensemble_Data/MFCC/models/')
mfcc_data['data_testing']= temp['data_testing']
mfcc_data['data_validation']= temp['data_validation']


#### Get data for wavelet model

In [ ]:
!pip install dtcwt

In [14]:
import librosa
from collections import Counter
import scipy
import dtcwt


trans = dtcwt.Transform1d(biort='antonini', qshift='qshift_d')

def calculate_entropy(list_values):
	counter_values = Counter(list_values).most_common()
	probabilities = [elem[1]/len(list_values) for elem in counter_values]
	entropy=scipy.stats.entropy(probabilities)
	return entropy

def calculate_statistics(list_values):
	n5 = np.nanpercentile(list_values, 5)
	n25 = np.nanpercentile(list_values, 25)
	n75 = np.nanpercentile(list_values, 75)
	n95 = np.nanpercentile(list_values, 95)
	median = np.nanpercentile(list_values, 50)
	mean = np.nanmean(list_values)
	std = np.nanstd(list_values)
	var = np.nanvar(list_values)
	rms = np.nanmean(np.sqrt(list_values**2))
	return [n5, n25, n75, n95, median, mean, std, var, rms]

def calculate_crossings(list_values):
	zero_crossing_indices = np.nonzero(np.diff(np.array(list_values) > 0))[0]
	no_zero_crossings = len(zero_crossing_indices)
	mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
	no_mean_crossings = len(mean_crossing_indices)
	return [no_zero_crossings, no_mean_crossings]

def get_features(list_values):
	entropy = calculate_entropy(list_values)
	crossings = calculate_crossings(list_values)
	statistics = calculate_statistics(list_values)
	return [entropy] + crossings + statistics

def extract_dtcwt(file_path:str):
    d, fs = librosa.load(file_path)
    forw = trans.forward(d, nlevels=17)
    features = []
    for coeff in forw.highpasses:
        temp = (np.abs(coeff.squeeze()))
        features += get_features(temp)

    features += get_features(forw.lowpass.squeeze())
    return features

def get_data_wavelet(training_percentaje=0.8, validation_percentaje=0.1, test_percentaje=0.1, data_path ='/content/drive/MyDrive/GTZAN/Ensemble_Data/audios/ensemble/'):
    return get_data(data_path, genres, extract_dtcwt, training_percentaje,validation_percentaje,test_percentaje)

wavalet_data = get_data_wavelet(1,0,0)

temp  = get_data_wavelet(0.7, 0.15, 0.15, '/content/drive/MyDrive/GTZAN/Ensemble_Data/audios/models/')
wavalet_data['data_testing']= temp['data_testing']
wavalet_data['data_validation']= temp['data_validation']

#### Get Conv1D data

In [ ]:
data_path = '/content/drive/MyDrive/GTZAN/Ensemble_Data/audios/models/'

# CARGAR UN ARCHIVO DE AUDIO
def load_audio_file(file_path):
    input_length = 660000  # This is 15 seconds with 44100 sample rate
    data = librosa.core.load(file_path, sr=22050)[0]  # We use librosa to load audio file with sample rate 22050
    if len(data) > input_length:
        data = data[:input_length]
    else:
        data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
    return data

def get_data_conv1D(training_percentaje=0.8, validation_percentaje=0.1, test_percentaje=0.1):
    return get_data(data_path, genres, load_audio_file,training_percentaje,validation_percentaje,test_percentaje)

conv1D_data = get_data_conv1D()

conv1D_data['data_training']['in'] = conv1D_data['data_training']['in']
conv1D_data['data_validation']['in'] = conv1D_data['data_validation']['in']
conv1D_data['data_testing']['in'] = conv1D_data['data_testing']['in']


#### Fill all datas

In [16]:
data = {
    'mfcc': mfcc_data,
    'wavalet': wavalet_data
}

### Import model.h5 for each model

In [21]:
import pickle


with open('/content/drive/MyDrive/Ensemble/dtcwt_rf.bin', 'rb') as archivo:
    dtcwt_rf_model = pickle.load(archivo)

mfcc_model = keras.models.load_model('/content/drive/MyDrive/Ensemble/mfcc_model.h5')
models = {
    'mfcc': mfcc_model,
    'wavalet':dtcwt_rf_model
}

### Train and Test (Ensemble Model)


#### Train

In [26]:
import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Dense

classes = len(genres)

def train():
    train_data = []
    models_predict = []
    models_predict_validation = []

    for key,model_ in models.items():
        models_predict.append(model_.predict(data[key]['data_training']['in']))
        models_predict_validation.append(model_.predict(data[key]['data_validation']['in']))

    #Cambiar filas por columnas para tener n datos de tamanno len(modelos), que cada uno seria un array con la salida de cada modelo evaluado en los datos
    stacked_predictions = np.column_stack(models_predict)
    stacked_predictions_val = np.column_stack(models_predict_validation)

    model = Sequential()
    model.add(Dense(100, input_dim=len(stacked_predictions[0]), activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(classes, activation='softmax'))

    model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy'])



    #se le pasa cualquier salida, ya que todas tendran igual tamanno e iguales
    model.fit(
        stacked_predictions, mfcc_data['data_training']['out'],
        validation_data=(stacked_predictions_val, mfcc_data['data_validation']['out']),
        epochs=epochs_,
        )

    return model

model = train()

1/1 [==============================] - 1s 865ms/step
Epoch 1/1000
5/5 [==============================] - 1s 53ms/step - loss: 2.3463 - accuracy: 0.1187 - val_loss: 2.2820 - val_accuracy: 0.2000
Epoch 2/1000
5/5 [==============================] - 0s 11ms/step - loss: 2.2647 - accuracy: 0.1750 - val_loss: 2.2404 - val_accuracy: 0.2000
Epoch 3/1000
5/5 [==============================] - 0s 10ms/step - loss: 2.2355 - accuracy: 0.2000 - val_loss: 2.2131 - val_accuracy: 0.2500
Epoch 4/1000
5/5 [==============================] - 0s 15ms/step - loss: 2.2064 - accuracy: 0.2875 - val_loss: 2.1855 - val_accuracy: 0.2500
Epoch 5/1000
5/5 [==============================] - 0s 13ms/step - loss: 2.1750 - accuracy: 0.2250 - val_loss: 2.1636 - val_accuracy: 0.1500
Epoch 6/1000
5/5 [==============================] - 0s 13ms/step - loss: 2.1484 - accuracy: 0.2500 - val_loss: 2.1444 - val_accuracy: 0.3000
Epoch 7/1000
5/5 [==============================] - 0s 13ms/step - loss: 2.1197 - accuracy: 0.3313 - 

#### Test

In [ ]:
def test():
    models_predict_test = []
    for key,model_ in models.items():
        models_predict_test.append(model_.predict(data[key]['data_testing']['in']))

    input = np.column_stack(models_predict_test)
    output = mfcc_data['data_testing']['out']

    score = model.evaluate(input, output, verbose=0)
    print('testing accuracy: ' + str(score[1]))

test()